# Importing python packages

In [1]:
import glob
from multiprocessing import Pool
import time
from tqdm import tqdm

# Importing SSPARQ setup

In [2]:
from parameters_py.config import (
					WAVEFORM_DIR,CATALOG_FILE,XML_DIR,SSPARQ_OUTPUT,num_processes
				   )

Reading configuration file: ./config_file.cnf


# Importing SSPARQ functions

In [3]:
from src.analysis import (
					aic_simple,find_orientation,Braunmiller_Pornsopin_algorithm,calculate_metrics
				   )

from src.utils import (
					quakeml_to_dataframe,moment_tensor_to_nodal_planes,calculate_plunge,mecclass,adjust_baz_for_ZEN,rms,energy
				   )

# Main program

## Example I: One station

### Reading catalog

In [4]:
cat = quakeml_to_dataframe(CATALOG_FILE)
cat.tail(2)

,time,latitude,longitude,depth,mag,magType,moment tensor
2048,2025-01-02 20:43:38.900,-21.70,-69.05,112.2,6.12,mw,"[-1.43e+18, -3.46e+17, -1.78e+18, -1.63e+17, 6..."
2049,2025-01-01 13:48:30.100,-56.44,-26.49,82.2,6.21,mw,"[1.56e+17, 2.13e+18, 2.2900000000000003e+18, -..."


### Reading station

In [8]:
STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))

In [9]:
if len(STATIONS_xml) < 2:
    STATIONS_xml = STATIONS_xml[0]

### Creating input list for one station

In [10]:
input_list = [[STATIONS_xml, WAVEFORM_DIR, event, SSPARQ_OUTPUT] for event in cat]

In [11]:
input_list = [
    [
        STATIONS_xml,
        WAVEFORM_DIR,
        row.depth,
        row.mag,
        row.magType,
        row.time,
        row.latitude,
        row.longitude,
        getattr(row, "moment tensor", []) if row._asdict().get("moment tensor") is not None else [],
        SSPARQ_OUTPUT,
    ]
    for row in cat.itertuples(index=False)
]

In [12]:
if input_list[0][8]:
    print('a')
else:
    print('b')

b


### Running multiprocessing function given a input list

In [13]:
start_time = time.time()

with Pool(processes=num_processes) as p:
    max_ = len(input_list)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(calculate_metrics,input_list):
            pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')

100%|██████████████████████████████████████| 2050/2050 [00:06<00:00, 335.36it/s]




--- 0.11 execution time (min) ---




## Example II: Multiple stations (determined by the number of files in the XML_DIR)

### Reading catalog

In [ ]:
cat = quakeml_to_dataframe(CATALOG_FILE)
cat.tail(2)

### Reading station

In [ ]:
STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))

In [ ]:
STATIONS_xml

### Creating input list for one station

In [ ]:
input_list = []
for xml_file in STATIONS_xml:
    input_list.append([xml_file,WAVEFORM_DIR,cat,SSPARQ_OUTPUT])

In [ ]:
print("Number of stations:",len(input_list))

### Running multiprocessing function given a input list

In [ ]:
start_time = time.time()

for sta_input in input_list:

    with Pool(processes=num_processes) as p:
        max_ = len(STATIONS_xml)
        with tqdm(total=max_) as pbar:
            for result in p.imap_unordered(calculate_metrics,sta_input):
                pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')